In [1]:
from ase import Atoms
from ase.visualize import view
import numpy as np

In [2]:
a_bcc = 3.11

In [3]:
bccTi = Atoms(cell=[a_bcc]*3, symbols='Ti2', scaled_positions=[[0, 0, 0], [0.5, 0.5, 0.5]], pbc=True)
view(bccTi, viewer='nglview')

In [4]:
positions = [np.array([i, 0, (6-2*i)%6]) for i in range(3)] + [np.array([i, 1, (6-2*i+3)%6]) for i in range(3)]
positions /= np.array([3, 2, 6])
twin_cell = Atoms(
    cell = a_bcc*np.array([(1+1+1)**0.5/2, (1+1+0)**0.5, (2**2+1+1)**0.5]),
    symbols = 'H6',
    scaled_positions = np.array(positions),
    pbc=True,
)
view(twin_cell*(1,1,1), viewer='nglview')

In [6]:
twin_cell.cell.cellpar()
twin_cell.write(filename=f'twin_112-111.vasp', format='vasp')

In [29]:
bbeta = twin_cell.cell.cellpar()[1]
vbeta = twin_cell.cell.cellpar()[0]
wbeta = twin_cell.cell.cellpar()[2]

shifts = np.array([
    np.array([0, 0, 0]),
    np.array([0, 0, 0]),
    np.array([0, 0, 0]),
    np.array([vbeta/2, 0, 0]),
    np.array([-vbeta/2, 0, 0]),
    np.array([-vbeta/2, 0, 0])
])

for zeta in np.linspace(0, 1, 43):
    twin = twin_cell.copy()
    pos = twin.get_positions()
    # apply shear
    twin.set_cell([[vbeta, 0, 0], [0, bbeta, 0], [-zeta*vbeta, 0, wbeta]], scale_atoms=True)
    # apply additional shuffling
    twin.set_positions(twin.positions + zeta*shifts)
    twin *= (3, 2, 2)
    twin.write(filename=f'str_z={zeta:.3f}.vasp', format='vasp')

## smaller cell

In [8]:
positions = [[0, 0, 0], [0.5, 0, 0.5],
             [0.25, 0.5, 0.25], [0.75, 0.5, 0.75]]
v = a_bcc*np.array([(1+1+1)**0.5/2, (1+1+0)**0.5, (2**2+1+1)**0.5])
twin_cell = Atoms(
    cell = [[v[0], 0, 0], [0, v[1], 0], [1/3*v[0], 0, 2/3*v[2]]],
    symbols = 'H4',
    scaled_positions = np.array(positions),
    pbc=True,
)
view(twin_cell*(3,2,2), viewer='nglview')

In [9]:
v = a_bcc*np.array([(1+1+1)**0.5/2, (1+1+0)**0.5, (2**2+1+1)**0.5])
positions = np.array([[0, 0, 0], [0.5, 0, 0.5],
             [0.25, 0.5, 0.25], [0.75, 0.5, 0.75]])
shifts = np.array([
    np.array([0, 0, 0]),
    np.array([0, 0, 0]),
    np.array([1/2, 0, 0]),
    np.array([-1/2, 0, 0])
])

for zeta in np.linspace(0, 1, 43):
    twin = Atoms(
        cell = [[v[0], 0, 0], [0, v[1], 0], [1/3*v[0]-zeta*2/3*v[0], 0, 2/3*v[2]]],
        symbols = 'H4',
        # apply shear+shuffling
        scaled_positions = positions+zeta*shifts,
        pbc=True,
    )    
    twin *= (3, 2, 2)
    twin.write(filename=f'str_z={zeta:.3f}.vasp', format='vasp')

In [10]:
twin_cell
twin_cell.write(filename=f'twin_112-111_Tobe.vasp', format='vasp')